In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import glob

import sklearn
from sklearn.metrics import confusion_matrix

In [ ]:
df = pd.read_csv(r"../input/cassava-leaf-disease-classification/train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df['image_id'] = df['image_id'].astype('string')
df['label'] = df['label'].astype('string')

In [ ]:
from keras.layers import Input,Lambda,Dense,Flatten
from keras.models import Model
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import warnings
from PIL import Image
warnings.filterwarnings("ignore",category=FutureWarning)
import os
import random
from keras.preprocessing.image import load_img
import tensorflow as tf
from PIL import Image
import cv2

In [ ]:
train_datagen = ImageDataGenerator(
                                     rescale=1./255,
                                     validation_split = 0.2,
                                     zoom_range = 0.2,
                                     horizontal_flip = True,
                                     vertical_flip = True,
                                     fill_mode = 'nearest',
                                     shear_range = 0.1,
                                     height_shift_range = 0.1,
                                     width_shift_range = 0.1)

train_generator = train_datagen.flow_from_dataframe(df,
                         directory = "../input/cassava-leaf-disease-classification/train_images",
                         subset = "training",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (400,400),
                         batch_size = 256,
                         class_mode = "categorical")

validation_datagen = ImageDataGenerator(rescale=1./255,
                                        validation_split = 0.2)

validation_generator = validation_datagen.flow_from_dataframe(df,
                         directory = "../input/cassava-leaf-disease-classification/train_images",
                         subset = "validation",
                         x_col = "image_id",
                         y_col = "label",
                         target_size = (400,400),
                         batch_size = 256,
                         class_mode = "categorical")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
plt.figure(figsize=(12,6))
sns.set_style('white')
sns.countplot(x="label",data=df)
plt.show()

In [ ]:
model = tf.keras.applications.mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, input_shape=(320, 320, 3))

In [ ]:
for layer in model.layers:
        layer.trainable=False

In [ ]:
x = model.outputs[0]
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
model_output = layers.Dense(5, activation='softmax')(x)
model = Model(inputs=model.input, outputs=model_output)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='Adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='recall', patience=2, mode='max', restore_best_weights=True, verbose=1
)

In [ ]:
history = model.fit(train_generator, epochs=10,callbacks=[early_stopping], validation_data=validation_generator)

In [ ]:
plt.plot(history.history['loss'],label='train loss')
plt.plot(history.history['val_loss'],label = 'val loss')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['accuracy'],label='train acc')
plt.plot(history.history['val_accuracy'],label='val acc')
plt.legend()
plt.show()

In [ ]:
ss = pd.read_csv(os.path.join("../input/cassava-leaf-disease-classification", "sample_submission.csv"))
ss

In [ ]:
preds = []

for image_id in ss.image_id:
    image = Image.open(os.path.join("../input/cassava-leaf-disease-classification",  "test_images", image_id))
    image = image.resize((320,320))
    image = np.expand_dims(image, axis = 0)
    preds.append(np.argmax(model.predict(image)))

ss['label'] = preds
ss

In [ ]:
ss.to_csv('submission.csv', index = False)